In [2]:
# Written by Elias Kraihanzel
# This is code for a sequential model that trains on 3000 images and tests the model against 3000 held out images

import os, cv2, random
import numpy as np
import pandas as pd
from PIL import Image, ImageFilter

from random import random
import math
from keras.utils.np_utils import to_categorical

import keras
import h5py

import matplotlib.pyplot as plt
from matplotlib import ticker
%matplotlib inline 

from keras.models import Sequential
from keras.layers import Input, Dropout, Flatten, Convolution2D, MaxPooling2D, Dense, Activation
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import np_utils

In [16]:
train_library_images_dir = 'data/train/library/'
train_campus_center_dir = 'data/train/campus_center/'
train_other_dir = 'data/train/other/'

test_library_images_dir = 'data/test/library/'
test_campus_center_dir = 'data/test/campus_center/'
test_other_dir = 'data/test/other/'

ROWS = 50
COLS = 50
CHANNELS = 1

train_library_img = [train_library_images_dir+i for i in os.listdir(train_library_images_dir)]
train_cc_img = [train_campus_center_dir+i for i in os.listdir(train_campus_center_dir)]
train_other_img = [train_other_dir+i for i in os.listdir(train_other_dir)]

test_lib_img = [test_library_images_dir+i for i in os.listdir(test_library_images_dir)]
test_cc_img = [test_campus_center_dir+i for i in os.listdir(test_campus_center_dir)]
test_other_img = [test_other_dir+i for i in os.listdir(test_other_dir)]
    
all_train = train_other_img + train_library_img + train_cc_img  
    
all_test = test_other_img + test_cc_img + test_lib_img 


print(len(all_train))
print(len(all_test))

3000
3000


In [5]:
def read_image(file_path):
    img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    return cv2.resize(img, (ROWS, COLS), interpolation=cv2.INTER_CUBIC)

In [6]:
# Collect the all images and store them in train_data
train_data = np.ndarray((len(all_train), CHANNELS, ROWS, COLS), dtype=np.uint8)

i = 0
for image_file in all_train:
    print(image_file)
    train_data[i] = read_image(image_file).T
    i = i+1

In [9]:
# Shape the train data properly
train_data = np.array(train_data).reshape(len(train_data), ROWS, COLS, -1)

In [10]:
# Create the labels for the train data
number_of_classes = 3;

labels = np.ndarray(len(all_train), np.int32)

i = 0
for file_name in all_train:
    if "other" in file_name:
        labels[i] = 0
    elif "library" in file_name:
        labels[i] = 1
    elif "campus_center" in file_name:
        labels[i] = 2
    i = i + 1

print(labels[800])
labels = to_categorical(labels,number_of_classes)
labels

0


array([[ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       ..., 
       [ 0.,  0.,  1.],
       [ 0.,  0.,  1.],
       [ 0.,  0.,  1.]])

In [17]:
test_data = np.ndarray((len(all_test), CHANNELS, ROWS, COLS), dtype=np.uint8)

i = 0
for image_file in all_test:
    print(image_file)
    test_data[i] = read_image(image_file).T
    i = i+1
    

data/test/other/0291da9d1fd5638a09c3a75912ec0cc8733a018f.jpg
data/test/other/08d8ba02a6cf0cccc82e380289009c45040ea3c4.jpg
data/test/other/0909922540e96479a9e11c6b696e17472b79fed4.jpg
data/test/other/0f1001a784b0beddd6af7149bb6ed7c23a77cdd0.jpg
data/test/other/109d3236cf73cbdd5fc733076746b62d19967c6b.jpg
data/test/other/11e3c2b25a0aa9df1e21e48353297b00a0121bcf.jpg
data/test/other/13c9c88d29aa5ff76f322a167a716a83e241d0b3.jpg
data/test/other/14c6eef6c4dd759c7d23d758f2a0f72e76b4d65b.jpg
data/test/other/158b40c3cf7ce668335a28cd8f5f1bb002bc528f.jpg
data/test/other/2013-09-24 20.44.26.jpg
data/test/other/2013-09-24 20.44.56.jpg
data/test/other/2017-04-29 15.41.07-1.jpg
data/test/other/2017-04-29 15.41.07-2.jpg
data/test/other/2017-04-29 15.41.08-1.jpg
data/test/other/2017-04-29 15.41.08-2.jpg
data/test/other/2017-04-29 15.41.10.jpg
data/test/other/2017-04-29 15.41.11-1.jpg
data/test/other/2017-04-29 15.41.11-2.jpg
data/test/other/2017-04-29 15.41.12-1.jpg
data/test/other/2017-04-29 15.41.12-2

In [19]:
# Shape the test data properly
test_data = np.array(test_data).reshape(len(test_data), ROWS, COLS, -1)

In [20]:
# Create the labels for the train data
number_of_classes = 3;

test_labels = np.ndarray(len(all_test), np.int32)

i = 0
for file_name in all_test:
    if "other" in file_name:
        test_labels[i] = 0
    elif "library" in file_name:
        test_labels[i] = 1
    elif "campus_center" in file_name:
        test_labels[i] = 2
    i = i + 1

    
print(all_test[0])
test_labels = to_categorical(test_labels,number_of_classes)
test_labels

data/test/other/0291da9d1fd5638a09c3a75912ec0cc8733a018f.jpg


array([[ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       ..., 
       [ 0.,  1.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  1.,  0.]])

In [21]:
#Sequential Model
# From https://github.com/transcranial/keras-js/blob/master/demos/notebooks/mnist_cnn.ipynb

model = Sequential()
model.add(Convolution2D(32, 3, 3, border_mode='valid', input_shape=(ROWS, COLS, CHANNELS), dim_ordering='tf'))
model.add(Activation('relu'))
model.add(Convolution2D(32, 3, 3, border_mode='valid', dim_ordering='tf'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), border_mode='valid', dim_ordering='tf'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(number_of_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adadelta(), metrics=['categorical_accuracy'])

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="valid", data_format="channels_last", input_shape=(50, 50, 1...)`
  """
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="valid", data_format="channels_last")`
  import sys
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(padding="valid", data_format="channels_last", pool_size=(2, 2))`
  if __name__ == '__main__':


In [ ]:
model.fit(train_data, labels, nb_epoch=200, batch_size=300)

Epoch 1/200
3000/3000 [==============================] - 9s - loss: 10.7400 - categorical_accuracy: 0.3337     
Epoch 2/200
3000/3000 [==============================] - 9s - loss: 10.7401 - categorical_accuracy: 0.3337     
Epoch 3/200
3000/3000 [==============================] - 10s - loss: 10.7400 - categorical_accuracy: 0.3337    
Epoch 4/200
3000/3000 [==============================] - 10s - loss: 10.7400 - categorical_accuracy: 0.3337    
Epoch 5/200
3000/3000 [==============================] - 10s - loss: 10.7400 - categorical_accuracy: 0.3337    
Epoch 6/200
3000/3000 [==============================] - 10s - loss: 10.7400 - categorical_accuracy: 0.3337    
Epoch 7/200
3000/3000 [==============================] - 10s - loss: 10.7400 - categorical_accuracy: 0.3337    
Epoch 8/200
3000/3000 [==============================] - 10s - loss: 10.7400 - categorical_accuracy: 0.3337    
Epoch 9/200
3000/3000 [==============================] - 10s - loss: 10.7400 - categorical_accuracy: 0.3

In [169]:
model.predict(train_data,batch_size = 300)

array([[  9.99998093e-01,   6.26120533e-09,   1.86009436e-06],
       [  9.99288142e-01,   7.04186095e-04,   7.80240498e-06],
       [  1.00000000e+00,   1.74315995e-09,   1.03557713e-13],
       ..., 
       [  6.67481070e-10,   1.00000000e+00,   1.51716661e-09],
       [  3.17472131e-06,   9.99946594e-01,   5.02018884e-05],
       [  7.88125362e-06,   9.99961615e-01,   3.05323447e-05]], dtype=float32)

In [171]:
model.evaluate(test_data, test_labels, batch_size=300)

3000/3000 [==============================] - 15s    


[0.54883885085582729, 0.88466666340827937]

In [172]:
model.save_weights('model.hdf5')

with open('model.json', 'w') as f:
    f.write(model.to_json())